Redes Neurais Convolucionais (CNNs ou ConvNets) são um tipo de arquitetura de rede neural profunda projetada especialmente para tarefas de processamento de imagens. Elas são inspiradas no funcionamento do sistema visual biológico dos animais e são amplamente utilizadas em visão computacional e tarefas de reconhecimento de padrões em imagens.

In [1]:
import numpy as np # manipular array multidimencionais
import matplotlib.pyplot as plt # criar graficos e plotar resultados
from keras.datasets import mnist, cifar10 # conjuntos de dados populares para classificar imagens, são usadas para testar redes neurais convolucionais

from keras.models import Sequential
#é uma pilha linear de camadas que você pode adicionar para construir sua rede neural.

from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
# Dense: Esta camada é uma camada totalmente conectada, comumente usada na parte final de redes neurais para classificação.
#Flatten: Esta camada é usada para transformar dados de entrada multidimensionais em uma forma unidimensional, comumente antes de camadas densas.
#Conv2D: Esta é uma camada de convolução 2D usada para extrair características de imagens.
#MaxPooling2D: Essa camada é usada para realizar o agrupamento máximo 2D, reduzindo a resolução espacial dos dados.

from keras.utils import to_categorical
# codificação one-hot de rótulos categóricos.

In [4]:
import os
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.applications import VGG16, InceptionV3, ResNet50, DenseNet121
from keras.optimizers import Adam
from sklearn.metrics import classification_report

from keras.models import Sequential
#é uma pilha linear de camadas que você pode adicionar para construir sua rede neural.

from keras.layers import Dense, Flatten, Dropout

# Defina o diretório raiz do seu conjunto de dados
train_dir = "C:\\Users\\cosmo\\Desktop\\ClassificadorDeImagens\\train"
test_dir = "C:\\Users\\cosmo\\Desktop\\ClassificadorDeImagens\\test"

# Crie um gerador de dados de imagem usando o flow_from_directory
datagen = ImageDataGenerator(rescale=1.0/255)  # Normaliza os valores dos pixels

# Carregue as imagens do diretório usando o flow_from_directory
image_size = (224, 224)  # Defina o tamanho desejado para as imagens
batch_size = 32  # Tamanho do lote
class_mode = 'binary'  # Modo de classificação

generator = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=class_mode,
    # color_mode='grayscale',  # Define o modo de cor para escala de cinza
)

# Verifique as classes disponíveis
class_names = list(generator.class_indices.keys())

# Visualize uma imagem do lote
images, labels = generator.next()
train_data, tmp_data, train_labels, tmp_labels = train_test_split(images, labels, test_size=0.3, random_state=42)
test_data, val_data, test_labels, val_labels = train_test_split(tmp_data, tmp_labels, test_size=0.5, random_state=42)

# Definindo o gerador de dados
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(train_dir,
                                              target_size=(224, 224),
                                              batch_size=32,
                                              class_mode='binary')

validation_generator = datagen.flow_from_directory(test_dir,
                                                   target_size=(224, 224),
                                                   batch_size=32,
                                                   class_mode='binary')

test_generator = datagen.flow_from_directory(test_dir,
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode='binary',
                                             shuffle=False)

# Definindo a função para criar modelos
def create_model(base_model):
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    return model

# Criando modelos com base nas arquiteturas pré-treinadas
vgg16_model = create_model(VGG16(weights='imagenet',
                                 include_top=False,
                                 input_shape=(224, 224, 3)))

inception_model = create_model(InceptionV3(weights='imagenet',
                                           include_top=False,
                                           input_shape=(224, 224, 3)))

resnet_model = create_model(ResNet50(weights='imagenet',
                                     include_top=False,
                                     input_shape=(224, 224, 3)))

densenet_model = create_model(DenseNet121(weights='imagenet',
                                          include_top=False,
                                          input_shape=(224, 224, 3)))

# Função para compilar e treinar o modelo
def compile_and_train(model, train_generator, validation_generator, epochs=10):
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['acc'])
    history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)
    return model, history

# Treinando os modelos
vgg16_model, vgg16_history = compile_and_train(vgg16_model, train_generator, validation_generator)
inception_model, inception_history = compile_and_train(inception_model, train_generator, validation_generator)
resnet_model, resnet_history = compile_and_train(resnet_model, train_generator, validation_generator)
densenet_model, densenet_history = compile_and_train(densenet_model, train_generator, validation_generator)

# Função para avaliar o modelo
def evaluate_model(model, test_generator):
    test_generator.reset()
    predictions = model.predict(test_generator)
    y_pred = np.round(predictions).astype(int)
    y_true = test_generator.classes
    report = classification_report(y_true, y_pred, target_names=['Normal', 'Covid-19'])
    print(report)

# Avaliando os modelos
print("VGG16:")
evaluate_model(vgg16_model, test_data)
print("InceptionV3:")
evaluate_model(inception_model, test_data)
print("ResNet50:")
evaluate_model(resnet_model, test_data)
print("DenseNet121:")
evaluate_model(densenet_model, test_data)


# Crie uma figura e um eixo
fig, ax = plt.subplots()

# Plote a primeira imagem do lote (agora em escala de cinza)
ax.imshow(np.squeeze(images[0]), cmap='gray')
ax.set_title(class_names[np.argmax(labels[0])])

# Exiba a imagem plotada
plt.show()


Found 18743 images belonging to 2 classes.
Found 18743 images belonging to 2 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Epoch 1/10
  3/586 [..............................] - ETA: 2:55:05 - loss: 1.0944 - acc: 0.4688